In [1]:
import nltk
from models import InferSent
import torch
import torch.nn as nn
from torchtext.data import Field, TabularDataset
import torch.optim as optim
import copy

In [2]:
ind = Field(sequential = False, use_vocab = False)

labels = Field(sequential = False, use_vocab = False)

sign = Field(sequential = False, use_vocab = False)

sent = Field(sequential = False, # if false, no tokenization is applied.
              use_vocab = True, #whether to use a vocab object. if false, the data is already numerical.
              #tokenize = tokenizer,
              lower=False)

fields = [('i', ind), ('l', labels), ('si', sign), ('s', sent)]

train_data, dev_data, test_data = TabularDataset.splits(path='/home/bozyurt20/Desktop/COMP542/Project/GLUE-baselines/glue_data/CoLA/', #open the files, read them and create the Tabular
                                              train = 'train.tsv',      #Dataset objects according to the format of the files
                                              validation = 'dev.tsv',
                                              test = 'test.tsv', 
                                              fields = fields, #and the field specifications.
                                              format = 'tsv'
                                              )

/opt/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/opt/anaconda3/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/opt/anaconda3/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further informa

In [12]:
list(train_data.s)

["Our friends won't buy this analysis, let alone the next one we propose.",
 "One more pseudo generalization and I'm giving up.",
 "One more pseudo generalization or I'm giving up.",
 'The more we study verbs, the crazier they get.',
 'Day by day the facts are getting murkier.',
 "I'll fix you a drink.",
 'Fred watered the plants flat.',
 'Bill coughed his way out of the restaurant.',
 "We're dancing the night away.",
 'Herman hammered the metal flat.',
 'The critics laughed the play off the stage.',
 'The pond froze solid.',
 'Bill rolled out of the room.',
 'The gardener watered the flowers flat.',
 'The gardener watered the flowers.',
 'Bill broke the bathtub into pieces.',
 'Bill broke the bathtub.',
 'They drank the pub dry.',
 'They drank the pub.',
 'The professor talked us into a stupor.',
 'The professor talked us.',
 'We yelled ourselves hoarse.',
 'We yelled ourselves.',
 'We yelled Harry hoarse.',
 'Harry coughed himself into a fit.',
 'Harry coughed himself.',
 'Harry coug

In [3]:
sentences = list(train_data.s)
labels = list(train_data.l)

dev_sentences = list(dev_data.s)
dev_labels = list(dev_data.l)

In [9]:
test_sentences = list(test_data.s)
test_labels = list(test_data.l)

AttributeError: 'Example' object has no attribute 's'

In [4]:
V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = 'fastText/crawl-300d-2M.vec'
infersent.set_w2v_path(W2V_PATH)

infersent.build_vocab(sentences, tokenize=True)

embeddings = infersent.encode(sentences, tokenize=True)
dev_embeddings = infersent.encode(dev_sentences, tokenize=True)

Found 5838(/5848) words with w2v vectors
Vocab size : 5838


/home/bozyurt20/Desktop/COMP542/Project/InferSent/models.py:207: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sentences = np.array(sentences)[idx_sort]


In [5]:
class Classifier(nn.Module):
    # define model elements
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer = nn.Linear(params_model['enc_lstm_dim']*2, 1)
        self.activation = nn.Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        X = self.layer(X)
        X = self.activation(X)
        return X
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Classifier()
model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [6]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(y_pred)

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [7]:
embeddings, labels = torch.tensor(embeddings, device=device), torch.tensor(list(map(int,labels)), device=device)
dev_embeddings, dev_labels = torch.tensor(dev_embeddings, device=device), torch.tensor(list(map(int,dev_labels)), device=device)

no_sentences = embeddings.shape[0]
dev_no_sentences = dev_embeddings.shape[0]

In [8]:
best_acc = 0.0
for e in range(1, 101):
    model.train()
    epoch_loss = 0.0
    epoch_acc = 0.0
    for i in range(no_sentences):
        #sent, label = sent.to(device), labels[i].to(device)
        optimizer.zero_grad()
        sent = embeddings[i,:]
        with torch.set_grad_enabled(True):
            y_pred = model(sent)      
            label = labels[i].type_as(y_pred)
            loss = criterion(y_pred, label.unsqueeze(0))
            acc = binary_acc(y_pred, label.unsqueeze(0))
            #print(y_pred)
            #print(acc)
            loss.backward()
            optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    model.eval()
    dev_epoch_loss = 0.0
    dev_epoch_acc = 0.0
    for i in range(dev_no_sentences):
        with torch.set_grad_enabled(False): 
            sent = dev_embeddings[i,:]
            y_pred = model(sent)      
            label = dev_labels[i].type_as(y_pred)
            loss = criterion(y_pred, label.unsqueeze(0))
            acc = binary_acc(y_pred, label.unsqueeze(0))
        
        dev_epoch_loss += loss.item()
        dev_epoch_acc += acc.item()
    if dev_epoch_acc > best_acc:
        best_acc = dev_epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())
      

    print(f'Epoch {e+0:03}: | trn-Loss: {epoch_loss/no_sentences:.5f} | trn-Acc: {epoch_acc/no_sentences:.3f}')
    print(f'Epoch {e+0:03}: | dev-Loss: {dev_epoch_loss/dev_no_sentences:.5f} | dev-Acc: {dev_epoch_acc/dev_no_sentences:.3f}')

Epoch 001: | trn-Loss: 0.60952 | trn-Acc: 68.998
Epoch 001: | dev-Loss: 0.62199 | dev-Acc: 69.128
Epoch 002: | trn-Loss: 0.60630 | trn-Acc: 70.120
Epoch 002: | dev-Loss: 0.62212 | dev-Acc: 69.128
Epoch 003: | trn-Loss: 0.60750 | trn-Acc: 69.770
Epoch 003: | dev-Loss: 0.62242 | dev-Acc: 68.936
Epoch 004: | trn-Loss: 0.60571 | trn-Acc: 70.132
Epoch 004: | dev-Loss: 0.62152 | dev-Acc: 69.128
Epoch 005: | trn-Loss: 0.60392 | trn-Acc: 70.518
Epoch 005: | dev-Loss: 0.62039 | dev-Acc: 69.319
Epoch 006: | trn-Loss: 0.60287 | trn-Acc: 70.717
Epoch 006: | dev-Loss: 0.62078 | dev-Acc: 69.223
Epoch 007: | trn-Loss: 0.60204 | trn-Acc: 70.577
Epoch 007: | dev-Loss: 0.62080 | dev-Acc: 69.223
Epoch 008: | trn-Loss: 0.60099 | trn-Acc: 70.799
Epoch 008: | dev-Loss: 0.62118 | dev-Acc: 69.128
Epoch 009: | trn-Loss: 0.60034 | trn-Acc: 70.986
Epoch 009: | dev-Loss: 0.62075 | dev-Acc: 69.223
Epoch 010: | trn-Loss: 0.59970 | trn-Acc: 70.553
Epoch 010: | dev-Loss: 0.62091 | dev-Acc: 69.223
Epoch 011: | trn-Los

Epoch 085: | trn-Loss: 0.53930 | trn-Acc: 81.219
Epoch 085: | dev-Loss: 0.61608 | dev-Acc: 68.744
Epoch 086: | trn-Loss: 0.53965 | trn-Acc: 81.055
Epoch 086: | dev-Loss: 0.61649 | dev-Acc: 68.552
Epoch 087: | trn-Loss: 0.53880 | trn-Acc: 81.277
Epoch 087: | dev-Loss: 0.61653 | dev-Acc: 68.456
Epoch 088: | trn-Loss: 0.53885 | trn-Acc: 81.172
Epoch 088: | dev-Loss: 0.61582 | dev-Acc: 68.552
Epoch 089: | trn-Loss: 0.53964 | trn-Acc: 81.043
Epoch 089: | dev-Loss: 0.61566 | dev-Acc: 68.840
Epoch 090: | trn-Loss: 0.53858 | trn-Acc: 81.289
Epoch 090: | dev-Loss: 0.61525 | dev-Acc: 68.648
Epoch 091: | trn-Loss: 0.53884 | trn-Acc: 81.160
Epoch 091: | dev-Loss: 0.61613 | dev-Acc: 68.936
Epoch 092: | trn-Loss: 0.53923 | trn-Acc: 81.020
Epoch 092: | dev-Loss: 0.61665 | dev-Acc: 67.689
Epoch 093: | trn-Loss: 0.53780 | trn-Acc: 81.336
Epoch 093: | dev-Loss: 0.61408 | dev-Acc: 68.648
Epoch 094: | trn-Loss: 0.53828 | trn-Acc: 81.254
Epoch 094: | dev-Loss: 0.61503 | dev-Acc: 68.360
Epoch 095: | trn-Los

In [10]:
model.load_state_dict(best_model_wts)

<All keys matched successfully>

In [11]:
best_acc = 0.0
optimizer.zero_grad()
model.eval()
epoch_loss = 0.0
epoch_acc = 0.0
for i in range(no_sentences):
    #sent, label = sent.to(device), labels[i].to(device)
    #
    sent = embeddings[i,:]
    with torch.set_grad_enabled(True):
        y_pred = model(sent)      
        label = labels[i].type_as(y_pred)
        loss = criterion(y_pred, label.unsqueeze(0))
        acc = binary_acc(y_pred, label.unsqueeze(0))
        #print(y_pred)
        #print(acc)
        loss.backward()
        optimizer.step()

    epoch_loss += loss.item()
    epoch_acc += acc.item()

dev_epoch_loss = 0.0
dev_epoch_acc = 0.0

for i in range(dev_no_sentences):
    with torch.set_grad_enabled(False): 
        sent = dev_embeddings[i,:]
        y_pred = model(sent)      
        label = dev_labels[i].type_as(y_pred)
        loss = criterion(y_pred, label.unsqueeze(0))
        acc = binary_acc(y_pred, label.unsqueeze(0))

    dev_epoch_loss += loss.item()
    dev_epoch_acc += acc.item()
if dev_epoch_acc > best_acc:
    best_acc = dev_epoch_acc
    best_model_wts = copy.deepcopy(model.state_dict())


print(f'Epoch {e+0:03}: | trn-Loss: {epoch_loss/no_sentences:.5f} | trn-Acc: {epoch_acc/no_sentences:.3f}')
print(f'Epoch {e+0:03}: | dev-Loss: {dev_epoch_loss/dev_no_sentences:.5f} | dev-Acc: {dev_epoch_acc/dev_no_sentences:.3f}')

Epoch 100: | trn-Loss: 0.62653 | trn-Acc: 59.128
Epoch 100: | dev-Loss: 0.62503 | dev-Acc: 67.593


In [22]:
y_pred

tensor([0.1700], device='cuda:0', grad_fn=<AddBackward0>)